# Control of a "spacecraft" platform with a reaction wheel

In [ ]:
# These are standard modules
import time
import numpy as np
from scipy import linalg
import sympy as sym
import matplotlib.pyplot as plt
from IPython.display import display, Latex, Markdown

# This is a custom interface to the pybullet simulator
import ae353_platform

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

Given a choice of ground roll angle $\phi$, the equations of motion are

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau - m_wgl\sin(q_1)\sin(\phi) \\
J_2 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau + \left( \dfrac{J_2}{J_1} \right) m_wgl\sin(q_1)\sin(\phi)
\end{aligned}
$$

where

* $q_1$ is the platform angle
* $\dot{q}_1$ is the platform velocity
* $v_2$ is the wheel velocity
* $\tau$ is the torque applied *to* the platform *by* the wheel

and

$$
J_1 = J_p + m_wl^2
\qquad
\qquad
J_2 = J_w
$$

and parameter values are defined as follows:

In [ ]:
# distance from platform axis to wheel axis
l = 1.

# radius of wheel
rw = 0.5

# mass of wheel
mw = 0.25

# moi of wheel
Jw = 0.5 * mw * rw**2

# mass of platform
mp = 12. * (1. - mw * l**2) / (3.**2 + 2.**2)

# moment of inertia of platform
Jp = (1. / 12.) * mp * (3.**2 + 2.**2)

# gravity
g = 9.81

# composite parameters
J1 = Jp + mw * l**2
J2 = Jw

Create an instance of the `Simulator` class, which is an interface to the [pybullet](http://pybullet.org) simulation engine.

There are two optional arguments:

* `display` (`True` or `False`) is whether or not to show the simulation window;
* `roll` (a floating-point number) is the angle of the ground.

You can also change `roll` on the fly by using the function `simulator.set_roll()`. If you use this function, you **must** call `simulator.reset(...)` (with appropriate initial conditions) and `controller.reset()` before running the simulator again, or your results may be unreliable.

In [ ]:
simulator = ae353_platform.Simulator(
    display=True,
    roll=(np.pi / 2),
)

Choose a default camera view (could also be `simulator.camera_topview()`).

In [ ]:
simulator.camera_sideview()

Define a controller that maps sensor measurements to actuator commands. The `run` function will be called 100 times per second (i.e., at 100 Hz) by the simulator.

In [ ]:
class Controller:
    def __init__(self):
        pass
    
    def reset(self):
        pass
    
    def run(
            self,
            t,
            platform_angle,
            platform_velocity,
            wheel_angle,
            wheel_velocity,
        ):
        
        wheel_torque = 0.
        
        return wheel_torque

Create an instance of the controller.

In [ ]:
controller = Controller()

Reset the simulator. Optional arguments allow you to specify the initial wheel angle and velocity. If the value of either argument is `None` (or if the argument is left unspecified), then its value will be chosen at random.

In [ ]:
simulator.reset(
    platform_angle=0.9,
    platform_velocity=0.,
    wheel_angle=0.,
    wheel_velocity=5.,
)

Reset the controller.

In [ ]:
controller.reset()

Run the simulator.

In [ ]:
data = simulator.run(
    controller,           # <-- required (an instance of your Controller class)
    max_time=5.0,         # <-- optional (how long you want to run the simulation in seconds)
    data_filename=None,   # <-- optional (name of file to which you want data saved, e.g., 'my_data.json')
    video_filename=None,  # <-- optional (name of file to which you want video saved, e.g., 'my_video.mov')
)

Get, show, and save a snapshot of the simulation after the last time step.

In [ ]:
# Get snapshot as height x width x 4 numpy array of RGBA values
rgba = simulator.snapshot()

# Display snapshot
plt.figure(figsize=(8, 8))
plt.imshow(rgba)

# Save snapshot
plt.imsave('my_snapshot.png', rgba)

Plot the results.

In [ ]:
t = data['t']
q1 = data['platform_angle']
v1 = data['platform_velocity']
v2 = data['wheel_velocity']
tau = data['wheel_torque']
tau_command = data['wheel_torque_command']

fig, (ax_q1, ax_v1, ax_v2, ax_tau) = plt.subplots(4, 1, figsize=(8, 8), sharex=True)

ax_q1.plot(t, q1, label='$q_1$', linewidth=4)
ax_v1.plot(t, v1, label='$v_1$', linewidth=4)
ax_v2.plot(t, v2, label='$v_2$', linewidth=4)
ax_tau.plot(t, tau, label=r'$\tau$', linewidth=4)
ax_tau.plot(t, tau_command, '--', label=r'$\tau_{commanded}$', linewidth=4)
ax_tau.plot(t, np.ones_like(t) * simulator.tau_max, ':', label=r'$\tau_{max}$', linewidth=4, color='C2', zorder=0)
ax_tau.plot(t, - np.ones_like(t) * simulator.tau_max, ':', linewidth=4, color='C2', zorder=0)

ax_q1.grid()
ax_q1.legend(fontsize=16)
ax_q1.tick_params(labelsize=14)

ax_v1.grid()
ax_v1.legend(fontsize=16)
ax_v1.tick_params(labelsize=14)

ax_v2.grid()
ax_v2.legend(fontsize=16)
ax_v2.tick_params(labelsize=14)

ax_tau.grid()
ax_tau.legend(fontsize=16)
ax_tau.tick_params(labelsize=14)

ax_tau.set_xlabel('time (s)', fontsize=20)
ax_tau.set_xlim([data['t'][0], data['t'][-1]])

fig.tight_layout()
plt.show()

Save the figure (without transparent background).

In [ ]:
fig.savefig('my_figure.png', facecolor='white', transparent=False)